```{contents}
:local:
:depth: 2
```

# Model Validation

## Learning Objectives

By the end of this chapter, you should be able to:

1. **Compute and interpret** multiple regression accuracy metrics (e.g.\ $r^2$, MAE, RMSE) and diagnose error structure with visual tools.  
2. **Apply** hold‑out, *k*-fold, and leave‑one‑out cross‑validation to estimate generalization error and discuss their trade‑offs.  
3. **Quantify prediction uncertainty** using residual analysis, resampling (bootstrapping), and Gaussian‐process regression.  
4. **Assess assumptions** such as homoskedasticity and data representativeness, and explain how violating them impacts error estimates.  
5. **Select appropriate validation and uncertainty techniques** for chemical‑engineering data sets of varying size and noise characteristics.

## Accuracy Metrics

Choosing appropriate accuracy metrics is a crucial step in evaluating regression models: these metrics quantify how closely predictions match observed data and help compare model performance in a standardized way. Before applying these metrics to a real dataset, we will illustrate their utility using a classic toy example—Anscombe’s quartet.

Anscombe’s quartet is a demonstration introduced by statistician Francis Anscombe in 1973. It consists of four small datasets, each with 11 (x, y) pairs, that share identical summary statistics—mean of x and y, variance, correlation coefficient, and parameters of a fitted linear regression—yet exhibit strikingly different patterns when plotted. This example underscores the importance of visualizing data before relying on numerical summaries alone. Below is a scatterplot for each dataset:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('../settings/plot_style.mplstyle')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Load Anscombe's quartet
df = sns.load_dataset('anscombe')  # columns: dataset, x, y

# Scatterplot for each dataset
fig, axes = plt.subplots(2, 2, figsize=(10, 8))
for ax, label in zip(axes.flatten(), ['I', 'II', 'III', 'IV']):
    sub = df[df['dataset'] == label]
    ax.scatter(sub['x'], sub['y'])
    ax.set_title(f'Dataset {label}')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
plt.tight_layout()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Compute metrics for Anscombe's quartet
rows = []
for label, group in df.groupby('dataset'):
    m, b = np.polyfit(group['x'], group['y'], 1)
    yhat = m * group['x'] + b
    rows.append({
        'Dataset': label,
        'R^2': r2_score(group['y'], yhat),
        'MAE': mean_absolute_error(group['y'], yhat),
        'RMSE': mean_squared_error(group['y'], yhat),
        'Max Error': np.max(np.abs(group['y'] - yhat))
    })
metrics_df = pd.DataFrame(rows)
metrics_df

```{note}
Notice that each of the four datasets yields nearly identical $r^2$ and RMSE, even though their scatterplots differ substantially. This illustrates how numerical summaries alone can mask important structural differences, and is a good reminder of the power of visualization and the need for using multiple metrics to evaluate models.
```

It is important to consider the context of a regression model and choose accuracy metrics that are relevant to its application. Below we introduce several commonly‑used options and illustrate them on Anscombe’s quartet.

### $r^2$ value

The $r^2$ metric varies from 0 to 1, with higher values corresponding to better models. It quantifies the fraction of variance in $y$ explained by the model, and $r^2$ will always increase (or stay constant) as you add more fitted parameters to the model. 


$$r^2 = 1 - \frac{\sum_i (y_i-\hat y_i)^2}{\sum_i (y_i-\bar y)^2}.$$

Conceptually, $r^2$ can be considered as a ratio between the sum-of-squares errors for the model (numerator) and the sum-of-squares errors for a "trivial" model that just always predicts the mean of the data (denominator).

```{admonition} Explanation
:class: note
A **negative $r^2$** can occur when the chosen model fits the data worse than the trivial model that always predicts the mean of $y$.  In other words, the sum‑of‑squares error of the model exceeds the total variance of the data.  This signals severe model mis‑specification or extrapolation outside the domain of validity. It is commonly encountered in cross validation, when the $r^2$ score is computed on a test or validation set that was not used to determine the parameters (and indicates that the model does not generalize beyond the training data).
```

### Mean absolute error (MAE)

MAE reports the average magnitude of errors in the original units of $y$, making it easy to interpret (“on average we miss by 0.3 bar”). Because the penalty is linear in $|y_i - \hat{y}_i|$, MAE is more robust to outliers than RMSE. As a loss, minimizing MAE corresponds to estimating the conditional median (for a constant model), so MAE emphasizes typical errors rather than occasional large ones.

$$\text{MAE} = \frac{1}{N}\sum_i |y_i-\hat y_i|.$$

### Root‑mean‑squared error (RMSE)

RMSE also has the units of $y$, but penalizes large errors more strongly due to squaring, so it is sensitive to outliers. If residuals are i.i.d. Gaussian with zero mean, RMSE is an estimate of the standard deviation of the errors and minimizing a loss function that uses RMSE aligns with maximum-likelihood estimation under that noise model. 
For the same dataset, RMSE $\geq$ MAE, with equality only when all absolute errors are equal.

$$\text{RMSE}=\sqrt{\frac{1}{N}\sum_i (y_i-\hat y_i)^2}. $$

### Error percentiles

Error percentiles give an alternative way to think about errors. If we define absolute errors $e_i = |y_i - \hat y_i|$, then the **$p$th error percentile** $Q_p$ is the value such that $p\%$ of the absolute errors are $\le Q_p$. Common choices are the **median** ($Q_{50}$, also called the *median absolute error*, MedAE), and tail summaries like $Q_{90}$, $Q_{95}$, or $Q_{99}$ to quantify “how bad things get” for the hardest cases.

Percentiles are in the same units as $y$, are easy to interpret ("90% of predictions are within 0.12 MPa"), and more robust to outliers than RMSE. They complement MAE/RMSE by separating **typical performance** (e.g., $Q_{50}$) from **tail risk** (e.g., $Q_{95}$). However, it is difficult to specifically optimize models for error percentiles, since they cannot be differentiated with standard methods.


### Maximum error

The worst‑case deviation, $\max_i |y_i-\hat y_i|$ , is useful when spec violations or safety margins matter (e.g., no prediction may err by more than 2 degrees C). It is extremely sensitive to a single outlier; practitioners often report it alongside a high quantile of the absolute error distribution (e.g., 95th percentile) to summarize tail risk more stably. Similar to percentiles, the max error is difficult to directly minimize.

### Parity plots

Plotting $y$ versus $\hat y$ gives a quick visual sense of bias (systematic over/under-prediction shifts points above/below the line) and spread (random error). Funnel shapes indicate heteroscedasticity (error grows with magnitude). Stratifying or coloring points by key features can reveal regime-dependent errors. Always pay attention to the scale of scatter plots, since they can sometimes be misleading if the range of the data is much larger than the typical errors (i.e., the errors "look small" on the scatter plot, but they may actually be large on an absolute scale).

Here we will make a parity plot for each model in Anscomb's quartet, assuming a simple linear regression model.

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from scipy.stats import linregress # a lightweight simple linear regression function

fig, axes = plt.subplots(1, 4, figsize=(15, 4))

for ax, label in zip(axes,['I', 'II', 'III', 'IV']):
    sub = df[df['dataset'] == label]
    x = sub['x']
    y = sub['y']

    ax.set_title(f'Dataset {label}')
    ax.set_xlabel('y')
    ax.set_ylabel('y_hat')
    slope, intercept, r, p, stderr = linregress(x, y)
    yhat = slope*x + intercept

    ax.scatter(y, yhat)
    ax.plot(y, y, 'k--') #an easy way to plot the 45 degree "parity" line

plt.tight_layout();

### Error histograms

Error (residual) histograms provide another perspective on model accuracy.  Whereas summary numbers like MAE or RMSE compress all errors into a single value, a histogram reveals *shape*: skewness, heavy tails, or multi‑modality in the residual distribution.

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(15, 4))

for ax, label in zip(axes,['I', 'II', 'III', 'IV']):
    sub = df[df['dataset'] == label]
    x = sub['x']
    y = sub['y']

    ax.set_title(f'Dataset {label}')
    ax.set_xlabel('y')
    ax.set_ylabel('y_hat')
    slope, intercept, r, p, stderr = linregress(x, y)
    yhat = slope*x + intercept
    residuals = y - yhat

    ax.hist(residuals, bins=5, edgecolor='k')
    ax.set_xlabel('Residual')
    ax.set_title(f'$\sigma = {residuals.std():.2f}$')

plt.tight_layout();

A *roughly Gaussian* residual histogram centered at zero indicates that errors are random and homoscedastic, supporting the use of aggregate metrics such as RMSE.  Skewed or heavy‑tailed residuals (as seen for Dataset IV) warn that single‑number metrics can hide important structure in the errors.


```{admonition} Exercise
:class: tip
Compute the MAE, RMSE, and maximum error for each dataset III in Anscomb's quartet with and without the outlier, and determine which metric is most sensitive to the outlier.
```

## Cross Validation

In the prior examples we computed error metrics for models that were trained with **all** available data.  However, what we really care about is *generalization*—how the model will perform on **new** data.  Gathering additional measurements is often impractical, so we **simulate** this situation through **cross‑validation**.  In cross‑validation some examples (the *test* set) are hidden while the model is fit to the remaining *training* examples; we then evaluate the loss on the hidden data to see how well the model predicts it.

There are many standard strategies:

- **hold‑out** – randomly leave out a percentage (commonly ≈30 %) of the data during training.
- **k‑fold** – split the data into *k* (typically 3–5) random, equally‑sized groups and train *k* times, holding each group out once.
- **leave‑p‑out** – leave *p* (often 1) samples out, evaluate the error on those *p* samples, and repeat for every possible size‑*p* subset.
- **bootstrapping** – sample with replacement to generate synthetic datasets of the same size, repeating many times.

Different techniques balance statistical robustness and computational effort.  Hold‑out is fast but can be sensitive to unlucky splits, especially for small datasets.  k‑fold alleviates that risk at the cost of *k* model fits.  Leave‑*p*‑out becomes expensive for *p > 1*.  **Doing some form of cross‑validation is almost always better than none.**

```{admonition}
:class: note
All cross‑validation techniques rely on the critical assumption that *the collected data are representative of future data*.  An example in chemical engineering would be building a model at one set of process conditions but then applying it under very different conditions. The CV accuracy will look deceptively good in that case, and the model will perform much worse when applied to the new conditions.
```

In this section, we will return to the ethanol IR spectrum dataset and demonstrate various types of cross validation.

In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Load data and extract the main peak region

df = pd.read_csv('data/ethanol_IR.csv')
x_all = df['wavenumber [cm^-1]'].values
y_all = df['absorbance'].values

x_peak = x_all[475:575]
y_peak = y_all[475:575]

fig, ax = plt.subplots()
ax.plot(x_peak, y_peak, '-', marker='.')
ax.set_xlabel('wavenumber [$cm^{-1}$]')
ax.set_ylabel('absorbance');

### Hold‑out cross validation

Hold-out cross validation is the simplest form, where you simply hide some subset of the data from the training. There are various ways of selecting which data to hold out, and different strategies will affect how the technique performs.

#### Deterministic every‑third‑point split

A quick deterministic variant of hold‑out is to **train on every Nth point** and test on the full dataset. This approach makes sense if you are trying to build a model that is capable of interpolating between known data, such as a model for replacing missing values. It is rarely used in practice, but is a useful conceptual demonstration.

In [ ]:
spacing = 3
sigma = 10
gamma = 1.0 / (2 * sigma ** 2)

x_train = x_peak[::spacing]
y_train = y_peak[::spacing]

# Radial basis function helper
def rbf(x_train, x_test=None, *, gamma=1.0):
    if x_test is None:
        x_test = x_train
    N, M = len(x_test), len(x_train)
    X = np.zeros((N, M))
    for i in range(N):
        for j in range(M):
            X[i, j] = np.exp(-gamma * (x_test[i] - x_train[j]) ** 2)
    return X

X_train = rbf(x_train, gamma=gamma)
model_rbf = LinearRegression().fit(X_train, y_train)
print(f"r^2 training = {model_rbf.score(X_train, y_train):.3f}")

X_all = rbf(x_train, x_test=x_peak, gamma=gamma)
print(f"r^2 testing  = {model_rbf.score(X_all, y_peak):.3f}")

fig, ax = plt.subplots()
ax.plot(x_peak, y_peak, 'o', markerfacecolor='none')
ax.plot(x_train, y_train, 'o')
ax.plot(x_peak, model_rbf.predict(X_all), '-')
ax.set_xlabel('wavenumber [$cm^{-1}$]')
ax.set_ylabel('absorbance')
ax.legend(['Original Data', 'Training Subset', 'Prediction']);

This is a kind of hold‑out because the model has never seen two‑thirds of the points.  The pattern in the residuals hints that our kernel width, $\sigma\$, may be sub‑optimal.

#### Random 60 / 40 split with `train_test_split`

Random splits avoid the structure of the deterministic split but introduce sampling variability.  By fixing `np.random.seed(0)` the split—and therefore the output—will be reproducible each time this cell is run. Random splits are more commonly used in practice, with 20-40% of the data typically being held out.

In [ ]:
from sklearn.model_selection import train_test_split
np.random.seed(0)

x_train, x_test, y_train, y_test = train_test_split(x_peak, y_peak, test_size=0.4)

sigma = 5
gamma = 1.0 / (2 * sigma ** 2)

model_rbf = LinearRegression().fit(rbf(x_train, gamma=gamma), y_train)
print(f"r^2 training = {model_rbf.score(rbf(x_train, gamma=gamma), y_train):.3f}")
print(f"r^2 testing  = {model_rbf.score(rbf(x_train, x_test=x_test, gamma=gamma), y_test):.3f}")

A parity plot makes the extrapolation failure obvious when $\sigma$ is ill‑chosen:

In [ ]:
fig, ax = plt.subplots()
ax.plot(y_test, model_rbf.predict(rbf(x_train, x_test=x_test, gamma=gamma)), 'o')
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()])
ax.set_xlabel('Actual')
ax.set_ylabel('Predicted');

### k‑fold Cross Validation

Hold‑out yields **one** estimate of test error; **k‑fold** repeats the procedure *k* times for more robust statistics. This reduces the chances that our results are based on a specific sample, but increases the computational cost.

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=0)
sigma = 100
gamma = 1.0 / (2 * sigma ** 2)

fig, ax = plt.subplots()
ax.plot(x_peak, y_peak, '-o', markerfacecolor='none')

r2_test = []
for train_idx, test_idx in kf.split(x_peak):
    x_tr, x_te = x_peak[train_idx], x_peak[test_idx]
    y_tr, y_te = y_peak[train_idx], y_peak[test_idx]

    model = LinearRegression().fit(rbf(x_tr, gamma=gamma), y_tr)
    r2 = model.score(rbf(x_tr, x_test=x_te, gamma=gamma), y_te)
    r2_test.append(r2)
    ax.plot(x_te, model.predict(rbf(x_tr, x_test=x_te, gamma=gamma)), '-')

ax.set_xlabel('wavenumber [$cm^{-1}$]')
ax.set_ylabel('absorbance')
ax.set_title(f'{kf.n_splits}-fold cross validation');
print(f"mean r^2 (test) = {np.mean(r2_test):.3f} ± {np.std(r2_test):.3f}")

 When the end‑points land in the test fold, the model must **extrapolate** and often fails catastrophically.  k‑fold CV lowers the risk of an **overly lucky** (or unlucky) split—but at the computational cost of *k* separate model fits.

````{admonition} Exercise
:class: note
**Task – 8‑fold MAE**  
Repeat the study using **8‑fold** CV and $\sigma=150$.  Compute the **mean** and **standard deviation** of the **mean‑absolute‑error (MAE)** across the folds.

```python
from sklearn.metrics import mean_absolute_error

kf = KFold(n_splits=8, shuffle=True, random_state=0)
mae = []
for tr, te in kf.split(x_peak):
    model = LinearRegression().fit(rbf(x_peak[tr], gamma=1/(2*150**2)), y_peak[tr])
    y_pred = model.predict(rbf(x_peak[tr], x_test=x_peak[te], gamma=1/(2*150**2)))
    mae.append(mean_absolute_error(y_peak[te], y_pred))
print(f"Mean of MAE: {np.mean(mae):.6f}")
print(f"Std  of MAE: {np.std(mae):.6f}")
```
````


### Leave‑One‑Out Cross Validation (LOO)

Leave‑one‑out is the limiting case of *k*-fold cross validation with *k = N* (the number of observations): each iteration trains on *N − 1* points and tests on the single left‑out point.  It provides an almost unbiased estimate of generalization error but can be computationally intensive for large datasets.

In [ ]:
from sklearn.model_selection import LeaveOneOut

loo = LeaveOneOut()
sigma = 10
gamma = 1.0 / (2 * sigma ** 2)

r2_loo = []
for tr_idx, te_idx in loo.split(x_peak):
    x_tr, x_te = x_peak[tr_idx], x_peak[te_idx]
    y_tr, y_te = y_peak[tr_idx], y_peak[te_idx]
    model = LinearRegression().fit(rbf(x_tr, gamma=gamma), y_tr)
    r2_loo.append(model.score(rbf(x_tr, x_test=x_te, gamma=gamma), y_te))

print(f"mean r^2 (LOO) = {np.mean(r2_loo):.3f}")
print(f"std  r^2 (LOO)  = {np.std(r2_loo):.3f}")

```{admonition}
:class: note
Because each LOO model is trained on nearly the entire dataset, its **bias** is low.  But the **variance** of the error estimate can be high: a single anomalous observation can strongly influence one iteration’s score.  Computationally, LOO requires *N* model fits, which is feasible for small‑to‑medium data but prohibitive for very large datasets.
```

```{admonition}
:class: tip
**Task**  
Investigate how the **test‑set fraction** influences the stability of performance estimates for the RBF model at $\sigma = 10$.

1. Choose three fractions: 20 %, 30 %, and 40 %.
2. For each fraction, perform at least **100** random splits with `train_test_split` (set `random_state=None` inside the loop).
3. Compute the test‑set $r^2$ for every split.
4. Visualize the resulting distributions (boxplots or histograms) side‑by‑side.

Which fraction yields the *lowest variance* in $r^2$?  Explain why smaller or larger test fractions may lead to more or less stable estimates.
```

## Quantifying Error and Uncertainty

In addition to model accuracy, it is often useful to have an estimate of *uncertainty*—a range that brackets the predictions we expect from future data.  We will go back to Anscombe’s quartet to illustrate these ideas.

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(15, 4))

yhat = m * x + b
axes[0].scatter(x, y1)
axes[0].plot(x, yhat, ls='-', color='k')
axes[1].scatter(x, y2)
axes[1].plot(x, yhat, ls='-', color='k')
axes[2].scatter(x, y3)
axes[2].plot(x, yhat, ls='-', color='k')
axes[3].scatter(x4, y4)
axes[3].plot(x4, m * x4 + b, ls='-', color='k');

### Standard Deviation of Error


One simple way of quantifying uncertainty is to assess the standard deviation of the *residuals* (model errors):

In [ ]:
error_stdev = np.std(y3 - yhat, ddof=2)
print(error_stdev)

Note that we used `ddof=2` here, since the linear model has two fitted parameters.  This yields an unbiased estimator of the population standard deviation, which is the same for each dataset in Anscombe’s quartet.

In [ ]:
x_dense = np.linspace(min(x), max(x), 200)
y_error = error_stdev

fig, axes = plt.subplots(1, 4, figsize=(15, 4))
for i, (xi, yi) in enumerate([(x, y1), (x, y2), (x, y3), (x4, y4)]):
    axes[i].scatter(xi, yi)
    axes[i].plot(x_dense, m * x_dense + b)
    axes[i].plot(x_dense, m * x_dense + b + y_error, ls='--', color='0.5')
    axes[i].plot(x_dense, m * x_dense + b - y_error, ls='--', color='0.5')
    axes[i].set_xlabel('x')
    axes[i].set_ylabel('y')
    axes[i].set_title(f'Dataset {i + 1}')

```{admonition}
:class: note
These $\pm1\,\sigma$ bands assume *homoskedastic*, Gaussian errors.  They clearly miss the outlier in Dataset&nbsp;4, showing that identical error bars do **not** suit every dataset.
```

### Discussion: Are these uncertainty bounds valid for all datasets in Anscombe’s quartet?

```{admonition}
:class: note
No.  The underlying error structure varies across the quartet—Dataset&nbsp;4, in particular, contains a large outlier—so constant-width error bands are inappropriate for all datasets.
```


### Resampling or "bootstrapping"


Another possibility that avoids homoskedastic assumptions is to **resample** the data to build empirical distributions of parameters that capture the deviations in the data.  A simple approach is to **sample with replacement** so that each re‑sample is slightly different.

In [ ]:
from numpy.random import choice  # randomly select items from a list

def bootstrap_linregress(x_all, y_all, N):
    m_list, b_list = [], []
    for _ in range(N):
        subset = choice(range(len(x_all)), size=len(x_all), replace=True)
        xprime = [x_all[j] for j in subset]
        yprime = [y_all[j] for j in subset]
        if np.std(xprime) > 0:
            m, b = np.polyfit(xprime, yprime, deg=1)
        else:
            m = 0
            b = np.mean(yprime)
        m_list.append(m)
        b_list.append(b)
    return m_list, b_list

anscombs = [[x, y1], [x, y2], [x, y3], [x4, y4]]
fig, axes = plt.subplots(1, 4, figsize=(15, 4))
N = 100
for i, (xi, yi) in enumerate(anscombs):
    m_list, b_list = bootstrap_linregress(xi, yi, N)
    axes[i].hist(m_list, bins=10, alpha=0.7)
    axes[i].set_xlabel('slope $m$')
    axes[i].set_title(f'Dataset {i + 1}')

You don't need to understand all the details of this code block, but should understand the general idea. Re-sampling is closely related to cross-validation. We hide some of the data from the model and see how the model changes. The difference is that we keep the **models**, rather than just analyzing the errors. Then we can use the models to give a range of estimates, or check the parameters of the models to see how much parameters change depending on the data.

This approach is very powerful because it allows us to get estimates of the prediction error as well as the error distribution of the parameters.

### Gaussian Process Regression

Gaussian process regression (GPR) is an extension of kernel methods that provides a *probabilistic* prediction complete with uncertainty.  A full treatment is beyond our scope, but we will briefly demonstrate it for the ethanol‑spectrum dataset:

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

x_peak = x_peak.reshape(-1, 1)
y_peak = y_peak.reshape(-1, 1)

x_train, x_test, y_train, y_test = train_test_split(x_peak, y_peak, test_size=0.4, random_state=0)

gpr = GaussianProcessRegressor(kernel=RBF(1), alpha=5e-6)
gpr.fit(x_train, y_train)

y_gpr, y_std = gpr.predict(x_peak, return_std=True)

fig, ax = plt.subplots()
ax.plot(x_train, y_train, 'o')
ax.plot(x_peak, y_gpr, '--')
ax.fill_between(x_peak[:, 0], y_gpr[:, 0] - y_std, y_gpr[:, 0] + y_std, alpha=0.2)
ax.set_xlabel('wavenumber [$cm^{-1}$]')
ax.set_ylabel('absorbance')
ax.legend(['Training Set', 'Prediction'])
ax.set_title('Gaussian Process Regression');

GPR is powerful for uncertainty estimation because it treats the unknown function as a **prior Gaussian process**—we assume any finite collection of function values follows a multivariate normal distribution whose *covariance* is governed by a **kernel** (covariance function).  The kernel encodes assumptions about smoothness, periodicity, and overall variation.

- **Kernel selection** — Common kernels include the squared‑exponential (RBF), Matérn, and periodic forms; sums or products of kernels can model more complex structure.  Choosing an inappropriate kernel can over‑smooth the data or yield wildly uncertain predictions.
- **Hyper‑parameter optimization** — Each kernel carries hyper‑parameters (e.g., length‑scale, variance).  Scikit‑learn maximizes the log‑marginal likelihood by default, but you can also tune hyper‑parameters via cross‑validation or Bayesian optimization.  Good uncertainty estimates depend critically on finding well‑calibrated values.
- **Computation** — Exact GPR scales as \$\mathcal{O}(N^3)\$ due to matrix inversion, so large datasets may require sparse approximations or inducing‑point methods.

For a candid discussion of the strengths and pitfalls of Bayesian model selection—including Gaussian processes—see the blog post **“Lies, damn lies, statistics, and *****Bayesian***** statistics”** ([https://kitchingroup.cheme.cmu.edu/blog/2025/06/22/Lies-damn-lies-statistics-and-Bayesian-statistics/](https://kitchingroup.cheme.cmu.edu/blog/2025/06/22/Lies-damn-lies-statistics-and-Bayesian-statistics/)).

```{admonition}
:class: tip
Use the **bootstrap_linregress** function with **$N = 500$** re‑samples on **Dataset 2** of Anscombe’s quartet.  Plot histograms of the bootstrapped slopes and intercepts, and report the **95 % empirical confidence interval** for each.
```



## Additional Reading

- {cite}`hastie2009` – **The Elements of Statistical Learning**, *Ch. 3* introduces model assessment and selection.  
- {cite}`goodfellow2016` – *Deep Learning*, Chap. 7 discusses regularization and validation.  
- {cite}`bishop2006` – *Pattern Recognition and Machine Learning*, Sect. 3.3 covers Gaussian processes.  
- {cite}`efron1994` – *An Introduction to the Bootstrap* for deeper insight into resampling methods.

```{{bibliography}}
```